In [6]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [7]:
! cp kaggle.json ~/.kaggle/

In [8]:
! chmod 600 ~/.kaggle/kaggle.json

In [9]:
!kaggle competitions download -c plant-seedlings-classification

 99% 1.68G/1.69G [00:59<00:00, 30.7MB/s]
100% 1.69G/1.69G [00:59<00:00, 30.4MB/s]


In [10]:
!unzip /content/plant-seedlings-classification.zip

Streaming output truncated to the last 5000 lines.
  inflating: test/ad12382d4.png      
  inflating: test/adb7a032c.png      
  inflating: test/ae90f2827.png      
  inflating: test/aecfaed64.png      
  inflating: test/aee6fa3df.png      
  inflating: test/af45e222a.png      
  inflating: test/afa446484.png      
  inflating: test/afcf6abd5.png      
  inflating: test/b026bf8ca.png      
  inflating: test/b03397525.png      
  inflating: test/b0acaff4a.png      
  inflating: test/b130a0632.png      
  inflating: test/b145ba9d4.png      
  inflating: test/b1cd2a91e.png      
  inflating: test/b215531dd.png      
  inflating: test/b2706e2b3.png      
  inflating: test/b29339405.png      
  inflating: test/b30ab4659.png      
  inflating: test/b31292706.png      
  inflating: test/b341d0aab.png      
  inflating: test/b39c71707.png      
  inflating: test/b3d6fdb80.png      
  inflating: test/b3e08b037.png      
  inflating: test/b47691c08.png      
  inflating: test/b4c3df835.png      

In [13]:
import os

In [15]:
label = list(os.listdir('/content/train'))

In [17]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

In [39]:
#Loading Data
data_path = '/content/train'
batch_size=128
resized_width=224
resized_height=224
classes = label

def load_split_train_test(datadir, valid_size = .2):
    train_transforms = transforms.Compose([
        transforms.Resize((resized_width,resized_height)),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))               
        ])
    test_transforms = transforms.Compose([
        transforms.Resize((resized_width,resized_height)),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))               
        ])
    train_data = torchvision.datasets.ImageFolder(
        root=datadir,  
        transform=train_transforms
    )
    test_data = torchvision.datasets.ImageFolder(
        root=datadir,
        transform=test_transforms
    )
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    from torch.utils.data.sampler import SubsetRandomSampler
    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    trainloader = torch.utils.data.DataLoader(
            train_data,
            batch_size=batch_size,
            num_workers=4,
#             shuffle=True,                                # Mutually exclusive with sampler
            sampler=train_sampler
    )
    testloader = torch.utils.data.DataLoader(
            test_data,
            batch_size=batch_size,
            num_workers=4,
#             shuffle=True,                               # Mutually exclusive with sampler
            sampler=test_sampler
    )
    return trainloader, testloader

trainloader, testloader = load_split_train_test(datadir=data_path, valid_size = .2)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [40]:
train_features, train_labels = next(iter(trainloader))

In [45]:
len(trainloader)

30

In [47]:
import matplotlib.pyplot as plt
import torch.nn as nn

In [49]:
flatten_model=nn.Flatten()

output = flatten_model(train_features[0])
output.shape

torch.Size([3, 50176])

In [77]:
class CNN_model(nn.Module):
  def __init__(self,input_shape, hidden_units,output_shape):
    super().__init__()
    
    self.block1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,out_channels =hidden_units,kernel_size=3,
                      stride =1,padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,out_channels =hidden_units,kernel_size=3,
                      stride =1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
    )

    self.block2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units,out_channels =hidden_units,kernel_size=3,
                      stride =1,padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,out_channels =hidden_units,kernel_size=3,
                      stride =1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
    )
    self.classifier = nn.Sequential(
      nn.Flatten(),
      nn.Linear(in_features = , out_features=output_shape)
      )

  def forward(self,x):
    x=self.block1(x)
    print(x.shape)
    x = self.block2(x)
    print(x.shape)
    x=self.classifier(x)
    return x

In [78]:
sample_model = CNN_model(input_shape=3,hidden_units=10,output_shape=len(label))

In [75]:
from torchsummary import summary

In [80]:
train_features[0].shape

torch.Size([3, 224, 224])

In [83]:
summary(sample_model,input_size=(3,56,56))

torch.Size([2, 10, 28, 28])
torch.Size([2, 10, 14, 14])


RuntimeError: ignored